In [96]:
from sly import Lexer, Parser

def read_scene():
    # Abre el archivo en modo lectura
    with open('Scene.txt', 'r') as archivo:
        # Lee el contenido del archivo
        a = archivo.read()
    contenido = a.split('\n')
    scene_options2 = sorted(contenido, key=len, reverse=True)
    Scene2='|'.join(scene_options2)
    return Scene2

def read_in_out():
    # Abre el archivo en modo lectura
    with open('InOut.txt', 'r') as archivo:
        # Lee el contenido del archivo
        a = archivo.read()
    contenido = a.split('\n')
    In_out_option = sorted(contenido, key=len, reverse=True)
    In_out2='|'.join(In_out_option)
    return In_out2

def read_moment():
    # Abre el archivo en modo lectura
    with open('Moment.txt', 'r') as archivo:
        # Lee el contenido del archivo
        a = archivo.read()
    contenido = a.split('\n')
    Moment_option = sorted(contenido, key=len, reverse=True)
    Moment2='|'.join(Moment_option)
    return Moment2

class MyLexer(Lexer):
    tokens={SEPARADOR, SCENE_OPTIONS, IN_OUT_OPTIONS, MOMENT_OPTIONS, NUMBER,TEXT}

    ignore=' \t\n'

    SCENE_OPTIONS=rf'{read_scene()}'
    IN_OUT_OPTIONS = rf'{read_in_out()}'
    MOMENT_OPTIONS = rf'{read_moment()}'
    NUMBER = r'\d+'
    SEPARADOR = r'\-|\-\-|/|\.'
    TEXT = r'[^\s\n\t]+'

class MyParser(Parser):
    tokens = MyLexer.tokens

    # Regla principal de la gramática que reconoce la estructura general
    @_('opcional_escena opcional_numero in_out_opcion texto_con_separadores moment_options')
    def entrada(self, p):
        return ( p.opcional_escena, p.opcional_numero, p.in_out_opcion, p.texto_con_separadores, p.moment_options)

    # Reglas para manejar la opción de tener o no SCENE_OPTIONS
    @_('SCENE_OPTIONS')
    def opcional_escena(self, p):
        return p.SCENE_OPTIONS

    @_('empty')
    def opcional_escena(self, p):
        return None

    # Reglas para manejar la opción de tener o no NUMBER
    @_('NUMBER SEPARADOR')
    def opcional_numero(self, p):
        return p.NUMBER

    @_('empty')
    def opcional_numero(self, p):
        return None

    # Regla para manejar IN_OUT_OPTIONS seguido de un separador
    @_('IN_OUT_OPTIONS SEPARADOR')
    def in_out_opcion(self, p):
        return p.IN_OUT_OPTIONS

    @_('TEXT secuencia_de_texto')
    def texto_con_separadores(self, p):
        return f"{p.TEXT} {p.secuencia_de_texto}"
    
    @_('SEPARADOR TEXT secuencia_de_texto ')
    def secuencia_de_texto(self, p):
        return f"{p.SEPARADOR} {p.TEXT} {p.secuencia_de_texto}"
    
    @_('TEXT secuencia_de_texto')
    def secuencia_de_texto(self, p):
        return f"{p.TEXT} {p.secuencia_de_texto}"

    @_('TEXT SEPARADOR')
    def secuencia_de_texto(self, p):
        return p.TEXT
    
    @_('IN_OUT_OPTIONS secuencia_de_texto')
    def secuencia_de_texto(self, p):
        return f"{p.IN_OUT_OPTIONS}{p.secuencia_de_texto}"
    
    @_('NUMBER secuencia_de_texto')
    def secuencia_de_texto(self, p):
        return f"{p.NUMBER} {p.secuencia_de_texto}"
    
    @_('MOMENT_OPTIONS')
    def moment_options(self, p):
        return p.MOMENT_OPTIONS

    # Regla para manejar casos vacíos (ausencia de SCENE_OPTIONS o NUMBER)
    @_('')
    def empty(self, p):
        return None

    # Manejo de errores sintácticos
    def error(self, p):
        pass

In [97]:
lexer = MyLexer()
parser = MyParser()


examples = [
        "ESC: 27. INT. POLICLÍNICO. CUARTO EVA (TERAPIA INTENSIVA) - TARDE"
    ]

for data in examples:
        token=lexer.tokenize(data)
        result = parser.parse(token)
        print(result)

('ESC:', '27', 'INT', 'POLICLÍNICO. CUARTO EVA (TERAPIA INTENSIVA)', 'TARDE')


In [98]:
class Scene(object):
    def __init__(self, number, in_out, place, moment, time, page,characters,continuity,text):
        self.number = number
        self.in_out = in_out
        self.place = place
        self.moment = moment
        self.time = time
        self.page = page
        self.characters=characters
        self.continuity=continuity
        self.text=text

In [107]:
data='''ESC: 31. INT. CASA ABUELA. CUARTO HUGO - TARDE
ROSTRO DE HUGO CON LA MISMA EXPRESIÓN.
HUGO
(angustiado)
¡¿Entiendes?! A lo nuestro no le
quiere poner nombre... Por si me
tiene que vender... O por si me
muero.
ABRE PLANO. DESCUBRIMOS QUE HUGO ESTÁ EN EL CUARTO, HABLANDO
CON SIMONE.
Simone está sentada en la cama, leyendo.
SIMONE
Hugo, ¿de qué tú estás hablando?
HUGO
Yo sé lo que te digo.
SIMONE
¿Tú no querías terminar con ella?
HUGO
(maravillado con el
recuerdo)
Tienes que verla en vestido, y con
ese pelo suelto... Es una belleza.
Simone, de manera muy circunspecta, cierra el libro.
Dando unas palmaditas en la cama, invita a Hugo a sentarse a
su lado. Hugo lo hace.
SIMONE
¿Puedo dejar caer sobre ti unas
palabras de sabiduría?
Hugo asiente.
Simone, repentina, agarra el libro y le da dos o tres
librazos a Hugo.
46
SIMONE
¡No seas superficial, chico! Cómo
vas a estar con alguien nada más
porque luce bien.
Simone se reclina, y vuelve a la lectura del libro.
HUGO
Es que no solo luce. También lo
está.
SIMONE
(lee)
Sé que eras débil; tan débil y tan
triste, que aún lo eras para el
amor.
HUGO
¿Qué tú me quieres decir, Simón?
SIMONE
(señalando el libro)
Yo no... Dulce María.
El sonido de Eva golpeando el plato metálico los interrumpe.
SIMONE
Abuela nos llama.
INICIO DE MONTAJE PROGRESIVO CON MÚSICA:
ESC: 32. INT. CASA ABUELA. COCINA-COMEDOR - TARDE
Eva, Simone, Hugo, Carmen, Ángela y Lucía. Todos sentados a
la mesa para comer.
Hugo mira a Carmen, pero esta se mantiene impasible.
La abuela le va a echar sal a la comida, y Simone se lo
impide.
ESC: 33. INT. CASA ABUELA. CUARTO EVA - NOCHE
Eva acostada en la cama. Simone se despide de ella con un
beso.
Simone recoge su almohada y su sábana y se va.
47
ESC: 34. INT. CASA ABUELA. CUARTO HUGO - NOCHE
Hugo y Simone durmiendo cómodos y desparramados en la cama.
Simone duerme con una sonrisa en el rostro.
ESC: 35. INT. CASA ABUELA. COCINA-COMEDOR - DÍA
Hugo jugando juego de rol con Eva, Carmen y Ángela. Tiran
dados y Hugo va contando historia. Tiene varios papeles con
estadísticas sobre la mesa.
Hugo gesticula como su tuviera una espada y estuviera
haciendo un cuento.
Ángela aplaude, emocionada.
Lucía, desde otro extremo de la mesa, los mira escéptica.
ESC: 36. INT. CASA ABUELA. SALA - DÍA
Eva abre la puerta de la casa y está Bernardo de pie, muy
bien vestido y con un ramo de flores en la mano.
Eva, con mala cara, le cierra la puerta en la cara.
ESC: 37. INT. CASA ABUELA. COCINA-COMEDOR - DÍA
Eva, Ángela, Lucía, Simone y Hugo, tejen cestos con mimbre.
Eva le muestra a Simone cómo tejer.
Carmen entra al comedor a dejar un poco de mimbre y sale.
Hugo se disculpa y sale detrás de Carmen. Señala su celular
como indicando que va a conectarse.
ESC: 38. EXT. CALLE DEL PUEBLO - DÍA
Hugo, Eva y Simone en una videollamada con los padres.
48
ESC: 39. INT. CASA ABUELA. COCINA-COMEDOR - DÍA
Hugo y Carmen cocinando.
Hugo le va a dar a probar algo en la boca a Carmen, y esta lo
mira seria. Niega levemente con la cabeza.
Hugo se vuelve. En la mesa están sentadas Ángela y Lucía, que
miran la escena, sonrientes.
Hugo y Carmen niegan, para disimular; como queriendo decir
que ahí no ocurre nada.
ESC: 40. INT. CASA ABUELA. SALA - DÍA
Simone, Eva y Hugo, recogen, organizan, y limpian la casa.
Eva abre la puerta de la casa. Está Bernardo, muy bien
vestido y con flores en la mano.
Eva nuevamente hace por cerrarle la puerta.
Simone se adelanta y lo evita. Toma a Bernardo de una mano, y
lo hace entrar.
ESC: 41. INT. CASA ABUELA. SALA - DÍA
Eva y Bernardo sentados en la sala hablando. Más bien el que
habla es Bernardo, Eva se mantiene media cerrada y seria.
En la mesita cerca de ellos, un ramo de flores puesto en
agua.
ESC: 42. INT./EXT. CASA ABUELA. PORTAL - TARDE
Hugo, Simone y Eva sentados en el portal, hablando entre
ellos, dándose sillón y saludando a los que pasan.
Hacen el mismo gesto que hicieran las tres abuelas en su
escena.
ESC: 43. INT. CASA JAIME. SALA - DÍA
Jaime haciendo fotos a alguien en el estudio. No sabemos a
49
quién se las hace.
Contra plano y vemos que es una sesión familiar de Hugo, Eva
y Simone.
Hugo tiene la espada de madera, y hace como si estuviera
combatiendo con el tigre del afiche.
Simone está avergonzada de su hermano.
ESC: 44. INT. CASA ABUELA. SALA - DÍA
Hugo y Simone colgando la foto familiar.
Eva, llena de felicidad, observa la foto.
FIN DE MONTAJE PROGRESIVO CON MÚSICA
ESC: 45. INT. CASA BERNARDO. SALA - DÍA
Bernardo y Simone organizan los libros.
Mientras lo hacen, conversan.
SIMONE
Convencí a mi abuela de que hagan
otras cosas... Sombreros, doyles,
portavasos... Pienso llevarme un
poco de mimbre cuando vuelva a mi
casa, y empezar mi propio negocio.
BERNARDO
Por favor, no dejes de leer poesía
por ganar un poco de dinero.
SIMONE
Claro que no, Bernardo.
BERNARDO
Pasa y no te das cuenta,
muchacha... Es bueno que quieras
ganarte lo tuyo, pero eso tiene que
ir a la par de todo lo demás...
Nunca descuides quien eres.
Simone encuentra entre los libros un álbum de fotos. Lo
hojea.
50
Bernardo lo mira, acongojado.
BERNARDO
Son mis dos hijos, con sus esposas
y mis nietos.
SIMONE
¿Hace mucho no los ves?
Un poco.
BERNARDO
Activo, Bernardo vuelve a acomodar los libros. Simone lo
contempla.
ESC: 46. EXT. CALLE DEL PUEBLO - DÍA
Eva sentada en un banco, conversa en videollamada con Aurora.
Cerca de ella, sentados en un contén, Hugo y Carmen
conversan.
AURORA
Mami, ¿por qué no nos dijiste nada
de tus problemas de salud?
EVA
Tú sabes que no me gusta
preocuparlos.
AURORA
No, vieja, tienes que decírnoslo.
EVA
Ustedes tampoco me preguntaron.
AURORA
Mami, no digas eso, que mi hermano
y yo nos preocupamos por ti.
ESC: 47. INT. CASA BERNARDO. SALA - DÍA
Bernardo habla mientras organiza. Continúa con Simone.
BERNARDO
Me dijeron para llevarme con
ellos... pero no quise...
51
ESC: 48. EXT. CALLE DEL PUEBLO - DÍA
Eva continúa hablando con Aurora.
AURORA
Tú no eres la misma de hace veinte
años... Ya tienes una edad en la
que no puedes estar sola.
ESC: 49. INT. CASA BERNARDO. SALA - DÍA
Continúa Bernardo y Simone conversando.
BERNARDO
...pero irme de aquí sería como
irme de mi vida.
Bernardo mira los libros.
ESC: 50. EXT. CALLE DEL PUEBLO - DÍA
Aurora y Eva.
AURORA
Cuando regresemos, te voy a mudar
cerca de nosotros.
ESC: 51. INT. CASA BERNARDO. SALA - DÍA
Bernardo contemplando los libros. Continúa conversando con
Simone.
BERNARDO
Después dejaron de insistirme, y
luego, no sé, supongo que solo
siguieron con sus vidas.
ESC: 52. EXT. CALLE DEL PUEBLO - DÍA
Aurora y Eva.
52
AURORA
O te vas con Enrique. Que te ha
insistido bastante para tenerte por
allá.
ESC: 53. INT. CASA BERNARDO. SALA - DÍA
Bernardo se sienta.
BERNARDO
(introspectivo)
Me esforcé mucho en que mis hijos
fueran grandes lectores... Los
castigaba, los obligaba... Y nunca
logré que se leyeran ni una página
de periódico.
ESC: 54. EXT. CALLE DEL PUEBLO - DÍA
Continúa la conversación entre Eva y Aurora.
EVA
Yo no me voy, Aurora... Y no tengo
cinco años para que me digas lo que
tengo que hacer.
ESC: 55. INT. CASA BERNARDO. SALA - DÍA
Simone se sienta en el piso junto a Bernardo.
SIMONE
¿Qué tiene eso que ver?
BERNARDO
Todo... Los hijos huyen de lo que
le imponen los padres... Y cuando
no se dan cuenta de lo que huyen,
pues huyen de los padres mismos.
ESC: 56. EXT. CALLE DEL PUEBLO - DÍA
Eva cuelga el celular.
53
Se queda observando la conversación en pantalla, y pincha el
último audio que está.
Escucha la voz de Simone.
SIMONE (O.S.)
(llorosa)
Mami, ¿por qué nos dejaste aquí? A
Eva no hay quien la soporte...
Estoy ahogada, mami. Este pueblo,
esta casa, me falta el aire...
No veo la hora de irnos de este
lugar... Si pudieran conseguir
dinero para llevarnos con ustedes
por este tiempo... Dime, mami... Yo
no puedo más.
Eva baja el celular. Pensativa, contempla el pueblo a su
alrededor. La vida que transcurre alrededor de ella con
normalidad. Todos pasan de un lugar a otro, y nadie parece
notarla. Se siente como si el banco fuese una isla.
Eva suspira... Toma aire antes de hablar.
EVA
Hugo, toma el teléfono... Y vamos
ya.
Hugo y Carmen se levantan para asistirla.
ESC: 57. INT. CASA ABUELA. COCINA-COMEDOR - TARDE
Lucía y Ángela tejen sentadas a la mesa. Eva también está
sentada, pero no hace nada.
Entran Simone y Bernardo. Bernardo trae un ramillete de
flores.
BERNARDO
Buenas tardes, Eva.
EVA
(muy seria)
Buenas tardes.
(mirando las flores)
Simón, ponlas en agua.
Simone toma las flores y las pone en un jarro con agua.
54
Entra Gonzalo con un file en las manos.
GONZALO
Buenas tardes, familia. ¿Cómo las
lleva el negocio?
SIMONE
Gonzalo, ya le hemos dicho que--
EVA
Pase, Gonzalo... Pase y siéntese.
GONZALO
En cuanto me envió el recado, vine
corriendo.
Ángela, Lucía, Simone y Bernardo se sorprenden.
Gonzalo pone el file frente a Eva y se lo abre. Luego se
sienta a la mesa.
GONZALO
Eva, fíjese bien en los números. Yo
sé que no se va a arrepentir.
Eva toma los papeles y comienza a leerlos.
SIMONE
Abuela. ¿Qué tú vas a hacer?
Vender.
EVA
SIMONE
¡¿Vender?! ¡¿El infarto también te
afectó la cabeza?!
ÁNGELA
Eva, ¿cómo vas a vender tu casa?
EVA
¿Me puedo quedar con ustedes unas
semanas?
Ángela asiente.
55
GONZALO
Eva, yo la puedo dejar aquí hasta
que le aparezca algo.
EVA
No, Gonzalo... Necesito el dinero
ya.
Gonzalo le pasa un bolígrafo a Eva.
GONZALO
(como quien no quiere las
cosas)
Si puede ir firmando a pie de cada
hoja.
SIMONE
¡Gonzalo, por favor!
(a Eva)
Abuela. Para esta locura.
Eva comienza a firmar los papeles.
Entran Hugo y Carmen desde la puerta del patio.
HUGO
¿Qué pasa?
Simone le quita el contrato a la abuela y lo rompe.
Gonzalo suspira. Con parsimonia saca otro contrato del file
que trae. Se lo pone delante a Eva.
LUCÍA
Está vendiendo la casa.
Simone va a quitarle el contrato, pero Eva la detiene.
EVA
Déjame, Simón... Con el dinero te
mando a ti, y a tu hermano, con sus
padres. Y que sean todos felices.
SIMONE
¡O sea, que no nos quieres aquí!
EVA
56
¡No digas eso!
SIMONE
(alterada)
¡Es verdad! Tienes el corazón de
piedra. No te gusta que te quieran.
Eva no le hace caso y sigue firmando.
Simón le arranca nuevamente el contrato, y lo rompe.
Gonzalo se da un spray de salbutamol. Saca otro juego de
contrato del file. Se lo va a pasar a Eva.
HUGO
¡¡¡Gonzalo, por favor!!!
Todos brincan, pues nadie se esperaba una reacción así de
Hugo.
HUGO
Abuela... Escúchame... Si tú
quieres vender la casa, véndela...
Pero no lo haga por nosotros.
Eva mira a Simón y a Hugo. Luego el contrato.
Toma el contrato y lo rompe.
Gonzalo se vuelve a dar otro toque de salbutamol.
Eva se levanta y se marcha hacia los cuartos. Todos quedan en
silencio.
ESC: 58. INT. CASA ABUELA. CUARTO EVA - DÍA
Eva está sentada en su cama.
Simone y Hugo entran al cuarto. Cierran la puerta al entrar.
SIMONE
Abuela.
57
EVA
(de espaldas a los nietos)
Me recuerdan tanto a su madre y a
Enrique... Mis niños... Cuando se
fueron esta casa se hizo enorme...
Ahora tengo miedo que, cuando
ustedes se vayan, vuelva a crecer.
Hugo se sienta junto a su abuela.
HUGO
Abuela...Nosotros no te vamos a
dejar sola.
Hugo abraza a la abuela.
Simón se sienta al otro lado de ella.
SIMONE
Pero déjate querer, vieja. Déjate
querer.
FUNDIDO A NEGRO
ESC: 59. INT. CASA ABUELA. CUARTO HUGO - DÍA
Hugo acostado en la cama. Pasa audios por WhatsApp para el
juego de rol. Hugo narra como lo hiciera en la primera
escena.
Simone, recostada cerca de él, lee un libro.
HUGO
(narra con atmósfera de
misterio)
Y cuando lo creen todo perdido, las
brujas del pantano deciden
perdonarles la vida... La hechicera
de hielo, el pícaro y el paladín,
comprenden que las brujas no son
malas, solo que llevan demasiado
tiempo solas, y han olvidado como
tratar a los viajeros de nobles
intenciones.
Simone mira a su hermano y entorna los ojos.
58
SIMONE
En serio, ahora todas las viejas de
tus cuentos son buenas.
Hugo le hace un gesto de que se calle.
Simone le hace una cara. Hugo se la devuelve.
Hugo continúa pasando audios al teléfono.
HUGO
(narra)
La noche y el frío están próximos.
La líder de las brujas se acerca a
ustedes y les dice con suaves
palabras:
Eva entra al cuarto.
EVA
Niños, la comida está servida.
Simone se levanta. Hugo suelta el teléfono.
SIMONE
Eh, ¿no vas a terminar?
Hugo chita, con un gesto de que le da igual.
HUGO
Después.
ESC: 60. INT. CASA ABUELA. COCINA-COMEDOR - DÍA
En la mesa todos almuerzan felices. Hugo y Carmen coquetean
delante de todos. Simone, Bernardo y Eva; Lucía y Ángela.
Todos comparten felices.
FIN.'''

In [108]:
def is_header(line):
    return parser.parse(lexer.tokenize(line))

In [109]:
scenes_headings = []
past_line=""
index_scene=1
text=""
old_result=None
        

page_content = data.split('\n')

for line in page_content:
    result = is_header(line.strip())
    if old_result is None:
        if result is None:
            past_line+=" "
            result2= is_header(past_line+line.strip())
            if result2 is None:
                past_line=line.strip()
            else:
                old_result=result2
                past_line=""
        else:
            old_result=result
            past_line=""
        continue
    elif result is None:
        past_line+=" "
        result3 = is_header(past_line+line.strip())
        if result3 is None:
            text+= "\n"+ past_line.strip()
            past_line=line.strip()
        else:
            if old_result[1] is None:
                scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
            else:
                scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
            past_line=""
            text=""
            old_result=result3
            index_scene+=1
    else:
        text+= "\n"+ past_line.strip()
        if old_result[1] is None:
            scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
        else:
            scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
        past_line=""
        text=""
        old_result=result
        index_scene+=1
            
if old_result != None:
    text+= "\n"+past_line.strip()
    if old_result[1] is None:
        scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
    else:
        scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))

In [110]:
for sc in scenes_headings:
    print(f'{sc.number} {sc.in_out} {sc.place} {sc.moment} ')

31 INT CASA ABUELA. CUARTO HUGO TARDE 
32 INT CASA ABUELA. COCINA-COMEDOR TARDE 
33 INT CASA ABUELA. CUARTO EVA NOCHE 
34 INT CASA ABUELA. CUARTO HUGO NOCHE 
35 INT CASA ABUELA. COCINA-COMEDOR DÍA 
36 INT CASA ABUELA. SALA DÍA 
37 INT CASA ABUELA. COCINA-COMEDOR DÍA 
38 EXT CALLE DEL PUEBLO DÍA 
39 INT CASA ABUELA. COCINA-COMEDOR DÍA 
40 INT CASA ABUELA. SALA DÍA 
41 INT CASA ABUELA. SALA DÍA 
42 INT./EXT CASA ABUELA. PORTAL TARDE 
43 INT CASA JAIME. SALA DÍA 
44 INT CASA ABUELA. SALA DÍA 
45 INT CASA BERNARDO. SALA DÍA 
46 EXT CALLE DEL PUEBLO DÍA 
47 INT CASA BERNARDO. SALA DÍA 
48 EXT CALLE DEL PUEBLO DÍA 
49 INT CASA BERNARDO. SALA DÍA 
50 EXT CALLE DEL PUEBLO DÍA 
51 INT CASA BERNARDO. SALA DÍA 
52 EXT CALLE DEL PUEBLO DÍA 
53 INT CASA BERNARDO. SALA DÍA 
54 EXT CALLE DEL PUEBLO DÍA 
55 INT CASA BERNARDO. SALA DÍA 
56 EXT CALLE DEL PUEBLO DÍA 
57 INT CASA ABUELA. COCINA-COMEDOR TARDE 
58 INT CASA ABUELA. CUARTO EVA DÍA 
59 INT CASA ABUELA. CUARTO HUGO DÍA 
60 INT CASA ABUELA. COCI